In [1]:
from tensorflow.keras import layers
import tensorflow as tf

from src.models.lstm import Seq2SeqAttentionLSTM, Seq2SeqLSTM
from src.data.scan import (
    MAX_SEQUENCE_LENGTH,
    IN_VOCAB_FILE,
    POS_VOCAB_SIZE,
    OUT_VOCAB_SIZE,
    IN_VOCAB_SIZE,
    POS_VOCAB_FILE,
    OUT_VOCAB_FILE,
)
from src.utils.constants import ACTION_INPUT_NAME, ACTION_OUTPUT_NAME, COMMAND_INPUT_NAME, POS_OUTPUT_NAME, POS_INPUT_NAME

In [2]:
in_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=IN_VOCAB_SIZE, standardize=None
)
pos_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=POS_VOCAB_SIZE, standardize=None
)
out_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=OUT_VOCAB_SIZE, standardize=None
)


in_vectorizer.set_vocabulary(IN_VOCAB_FILE)
pos_vectorizer.set_vocabulary(POS_VOCAB_FILE)
out_vectorizer.set_vocabulary(OUT_VOCAB_FILE)

out_voc = out_vectorizer.get_vocabulary()
pos_voc = pos_vectorizer.get_vocabulary()

In [3]:
model_in = "<sos> jump and run <eos>"
model_pos = "NOUN CONJ VERB"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
model_pos_vec = pos_vectorizer(tf.convert_to_tensor([model_pos, model_pos]))

In [16]:
hidden_size = 200
hidden_layers = 1
include_pos_tag = "input"
use_attention = True

Model = Seq2SeqAttentionLSTM if use_attention else Seq2SeqLSTM

model = Model(
    hidden_size=hidden_size,
    hidden_layers=hidden_layers,
    include_pos_tag=include_pos_tag,
    teacher_forcing=0,
)

pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)
checkpoint_path = f"snap\h_size({hidden_size})-h_layers({hidden_layers})-dropout(0.1)-pos({include_pos_tag}){'-attention' if use_attention else ''}/best_action_accuracy"
model.load_weights(checkpoint_path)

In [5]:
model_in = "<sos> jump and run <eos>"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))

In [31]:
model_in = "<sos> run around left <eos>"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)

actions = tf.argmax(pre_res[ACTION_OUTPUT_NAME][0], axis=-1)
# pos = tf.argmax(pre_res[POS_OUTPUT_NAME][0], axis=-1)

actions = " ".join([out_voc[i] for i in actions]).strip()
# pos = " ".join([pos_voc[i] for i in pos]).strip()

print(actions)
# print(pos)

<sos> I_TURN_LEFT I_RUN I_RUN I_RUN I_RUN <eos>
